this notebook analyzes https://github.com/leelabcnbc/thesis-yimeng-v2/blob/1874c2e8b68989e1faff68d7044f052cc8057569/scripts/feature_extraction/gaya/maskcnn_polished_with_rcnn_k_bl/submit_20200523.py

In [1]:
from os.path import dirname, relpath, realpath, join, exists

import h5py

from numpy.linalg import norm
import numpy as np

from thesis_v2 import dir_dict
from thesis_v2.submission import utils
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200523_gaya_feature_extraction_generator,
    script_keygen,
    gen_feature_extraction_global_vars
)
from json import dump, load

In [2]:
def generate_diff_vec(*, tensor_list, acc_mode, max_len):
    assert len(tensor_list) > 1
    
    if acc_mode=='instant':
        pass
    elif acc_mode == 'cummean':
        tensor_list_new = []
        for i in range(len(tensor_list)):
            tensor_list_new.append(np.mean(np.asarray(tensor_list[:i+1]), axis=0))
        tensor_list = tensor_list_new
    else:
        raise ValueError
    
    tensor_shape = tensor_list[0].shape
    # get norm diff
    diff_vec = np.full((max_len,), fill_value=np.nan, dtype=np.float64)
    
    for i in range(len(tensor_list)-1):
        vec_prev = tensor_list[i]
        vec_now = tensor_list[i+1]
        diff_this = norm(vec_now.ravel()-vec_prev.ravel())/norm(vec_prev.ravel())
        diff_vec[i] = diff_this
    return diff_vec.tolist()

def load_all():
    
    batch_key='gaya/20200523'
    global_vars_for_feature_extraction = gen_feature_extraction_global_vars(key=batch_key)
    
    for idx, param_dict in enumerate(explored_models_20200523_gaya_feature_extraction_generator()):
        
        cls = param_dict['rcnn_bl_cls']
        if cls <= 1:
            continue
        
        key_this = script_keygen(**param_dict)        
        h5file = join(global_vars_for_feature_extraction['feature_file_dir'], key_this+'.hdf5')
        
        auxfile = join(global_vars_for_feature_extraction['feature_file_dir'], key_this+'.json')
        
        if not exists(auxfile):
            with h5py.File(h5file, 'r') as f:
                g = f['test']
                resp_neurons = [g[f'1.{x}'][()] for x in range(cls)]
                diff_vec_neurons = generate_diff_vec(
                    tensor_list = resp_neurons,
                    acc_mode='instant',
                    max_len = 6,
                )

                resp_map = [g[f'0.{x}'][()] for x in range(cls)]
                diff_vec_map_instant = generate_diff_vec(
                    tensor_list = resp_map,
                    acc_mode='instant',
                    max_len = 6,
                )

                diff_vec_map_cummean = generate_diff_vec(
                    tensor_list = resp_map,
                    acc_mode='cummean',
                    max_len = 6,
                )
            # write aux file.
            with open(auxfile, 'wt', encoding='utf-8') as f_aux:
                dump({
                    'test': {
                        'diff_final_act': diff_vec_neurons,
                        'diff_bl_stack_instant': diff_vec_map_instant,
                        'diff_bl_stack_cummean': diff_vec_map_cummean,
                    }
                },
                    f_aux,
                )
        with open(auxfile, 'rt', encoding='utf-8') as f_aux:
            aux_obj = load(f_aux)
        
        if idx % 10 == 0:
            print(idx)
            print(aux_obj)
load_all()

10
{'test': {'diff_final_act': [0.05621732026338577, nan, nan, nan, nan, nan], 'diff_bl_stack_instant': [0.4065404534339905, nan, nan, nan, nan, nan], 'diff_bl_stack_cummean': [0.20327021181583405, nan, nan, nan, nan, nan]}}
20
{'test': {'diff_final_act': [0.08429772406816483, 0.04408401995897293, nan, nan, nan, nan], 'diff_bl_stack_instant': [0.6494524478912354, 0.3391950726509094, nan, nan, nan, nan], 'diff_bl_stack_cummean': [0.3247262239456177, 0.1697843372821808, nan, nan, nan, nan]}}
30
{'test': {'diff_final_act': [0.052190858870744705, 0.03088890016078949, 0.025772368535399437, nan, nan, nan], 'diff_bl_stack_instant': [0.41594988107681274, 0.26052936911582947, 0.21657204627990723, nan, nan, nan], 'diff_bl_stack_cummean': [0.20797494053840637, 0.12558889389038086, 0.09534070640802383, nan, nan, nan]}}
50
{'test': {'diff_final_act': [0.06336989253759384, nan, nan, nan, nan, nan], 'diff_bl_stack_instant': [0.5049763917922974, nan, nan, nan, nan, nan], 'diff_bl_stack_cummean': [0.25